# Derivadas superiores

Hasta ahora hemos visto que, usando diferenciación automática, podemos calcular la derivada de funciones de una variable esencialmente con un error del orden del epsilon de la máquina.

La pregunta que abordaremos ahora, es cómo hacer para calcular la segunda derivada, y derivadas de orden superior.

Una posibilidad, específica para el caso de la segunda derivada, es proceder como en el caso anterior, es decir, definir una *terna ordenada* donde la primer componente es el valor de la función en $x_0$, i.e., $f(x_0)$, el de la segunda es el valor de la primer derivada $f'(x_0)$, y la tercer componente tiene el valor de la segunda derivada, $f^{(2)}(x_0) = f''(x_0)$. 


Con esta definición, las operaciones aritméticas vienen dadas por:

\begin{eqnarray}
\vec{u} + \vec{v} & = & (u + v, \quad u'+ v', \quad u''+v''),\\
\vec{u} - \vec{v} & = & (u - v, \quad u'- v', \quad u''-v''),\\
\vec{u} \times \vec{v} & = & (u v, \quad u v' + u' v, \quad u v'' + 2 u' v' + u'' v),\\
\frac{\vec{u}}{\vec{v}} & = & \Big( \frac{u}{v}, \quad \frac{u'-( u/v) v'}{v}, \quad 
\frac{u'' - 2 (u/v)' v' - (u/v)v'' }{v}\Big).\\
\end{eqnarray}

Claramente, este proceso es muy ineficiente para derivadas de orden aún más alto, dado que las expresiones se complican y es fácil cometer errores.

# Series de Taylor

El punto importante a recordar, es que las derivadas de orden superior de una función $f(x)$ en un punto $x_0$ están contenidas en el desarrollo de Taylor de la función entorno a ese punto. La suposición importante en esto es que $f(x)$ es suficientemente suave; por simplicidad supondremos que $f(x)$ es ${\cal C}^\infty$ y que estamos suficientemente cerca del punto $x_0$, i.e., $|x-x_0|\ll 1$. 


La serie de Taylor de $f(x)$ viene dada por

\begin{eqnarray}
f(x) & = & f(x_0) + f^{(1)}(x_0) (x-x_0) + \frac{f^{(2)}(x_0)}{2!} (x-x_0)^2 + \dots + \frac{f^{(k)}(x_0)}{k!} (x-x_0)^k + \dots,\\
& = & f_{[0]} + f_{[1]} (x-x_0) + f_{[2]} (x-x_0)^2 + \dots + f_{[k]} (x-x_0)^k + \dots,\\
\end{eqnarray}

donde los coeficientes *normalizados* de Taylor $f_{[k]}$ que aparecen en la segunda línea de la ecuación anterior se definen como

\begin{equation}
f_{[k]} = \frac{f^{(k)}(x_0)}{k!} = \frac{1}{k!}\frac{{\rm d}^k f}{{\rm d}x^k}(x_0).
\end{equation}



Vale la pena **enfatizar** que la expresión anterior es exacta en tanto que la serie **no** sea truncada. En el caso de que la serie truncada a orden k, el [teorema de Taylor](https://en.wikipedia.org/wiki/Taylor%27s_theorem) asegura que el residuo (error del truncamiento) se puede escribir como:

\begin{equation}
{\cal R_{k}} = \frac{f^{(k+1)}\,(\xi)}{(k+1)!} (x-x_0)^{k+1},
\end{equation}

donde $\xi$ es un punto que pertenece al interval $[x_0,x]$.


Si la serie es truncada, la aproximación es un polinomio de orden $k$ (grado máximo es $k$) en $x$. Dado que los polinomios en una variable están definidos por $k+1$ coeficientes, entonces pueden ser mapeados a vectores en $\mathbb{R}^{k+1}$. 

Las operaciones aritméticas, en este caso, vienen dadas por:

\begin{eqnarray}
(f+g)_{[k]} & = & f_{[k]} + g_{[k]} ,\\
(f-g)_{[k]} & = & f_{[k]} - g_{[k]} ,\\
(f \cdot g)_{[k]} & = & \sum_{i=0}^k f_{[i]} \,g_{[k-i]} \, ,\\
\Big(\frac{f}{g}\Big)_{[k]} & = & \frac{1}{g_{[0]}}
\Big( f_{[k]} - \sum_{i=0}^{k-1} \big(\frac{f}{g}\big)_{[i]} \, g_{[k-i]} \Big) . \\
\end{eqnarray}

### Ejercicio

Implementen una nueva estructura paramétrica (`type`) que defina el tipo `Taylor`, donde el parámetro debe ser un subtipo de `Number`. Definan métodos que implementen las operaciones aritméticas básicas (`+`, `-`, `*`, `/`) y la igualdad (`==`). Esto deberá ser incluido en un módulo.

Incluyan pruebas (en el archivo "runtests.jl") para cada uno de los métodos que implementen.


In [2]:
"""Definición de polinomios de Taylor, parametrizados por un subtipo de Number.
"""
type Taylor{T<:Number}
    # código: 
    pol :: Array{T}
end

In [3]:
import Base.convert
import Base.promote_rule
import Base.+

convert{T<:Number}(::Type{Taylor{T}}, a::T) = Taylor(a)

promote_rule{T<:Number, S<:Number}(::Type{Taylor{T}}, ::Type{S}) = Taylor{(promote_type)(T,S)}


promote_rule (generic function with 125 methods)

In [4]:
Taylor(a::Complex) = Taylor ([a])


Use "Taylor(" instead.


Taylor{T<:Number}

In [5]:
"""Definimos una función que te regresa el grado máximo entre polinomios de Taylor para permitir las operaciones entre polinomios.
De igual manera saca el grado de un solo Taylor.
"""
function gradomax(a::Taylor,b::Taylor)
    return max(length(a.pol),length(b.pol))
end
gradomax(a::Taylor) = length(a.pol)

gradomax (generic function with 2 methods)

In [6]:
"""Aquí definimos una función que promueve dos polinomios a un grado común.
"""
function prom(a::Taylor,b::Taylor = a)
    q = [a.pol; fill(0,gradomax(a,b)-gradomax(a))]
    return Taylor(q)
end

prom (generic function with 2 methods)

In [7]:
# Polinomio de Taylor de una constante.

Taylor(a::Number) = Taylor([a])

Taylor{T<:Number}

In [8]:
import Base: +, -, *, ^, /, ==

# Aquí se implementan los métodos necesarios para la suma
+(a::Taylor, b::Taylor) = Taylor(prom(a,b).pol+prom(b,a).pol)
+(a::Taylor, k::Number) = a + Taylor(k)
+(k::Number, a::Taylor) = a + Taylor(k)

# Aquí los necesarios para la resta
-(a::Taylor, b::Taylor) = Taylor(prom(a,b).pol-prom(b,a).pol)
-(a::Taylor, k::Number) = a - Taylor(k)
-(k::Number, a::Taylor) = Taylor(k) - a
-(a::Taylor) = Taylor(-a.pol)

# Para la multiplicación de polinomios se optó por definir explícitamente la función.
function *(a::Taylor, b::Taylor)
    n = gradomax(a)+ gradomax(b)-1;
    r = Taylor(zeros(n));
    
    A = prom(a,r);
    B = prom(b,r)
    
    for k = 0:n-1
        suma = 0;
        
        for j = 0:k
            suma += A.pol[j+1]*B.pol[k-j+1];
        end
        r.pol[k+1] = suma;

    end
    
    return r
end
*(a::Taylor, k::Number) = Taylor(k*a.pol)
*(k::Number, a::Taylor) = Taylor(k*a.pol)

# Para la división se optó por un procedimiento similar.


# Igualdad
==(a::Taylor, b::Taylor) = a.pol == b.pol

== (generic function with 110 methods)

Para la división se optó por un procedimiento similar, pero se obtuvieron los términos de foma iterada. Se definió la función $h(x)=\frac{f(x)}{g(x)}$. Con esto $f(x)=h(x)*g(x)$, por lo que ahora se puede escribir el polinomio de Taylor (truncado) de f en términos del producto de los de h y g. Esto está dado por 

\begin{eqnarray}
f_{[k]}& = &(h \cdot g)_{[k]} & = & \sum_{i=0}^k h_{[k-i]} \,g_{[i]} \, \\
\end{eqnarray}

Para encontrar el primer término del polinomio de h, se separa la suma de la manera siguiente:


\begin{eqnarray}
f_{[k]}& = &(h \cdot g)_{[k]} & = & \ g_{[s]}h_{[k-s]}\ + \sum_{i=s+1}^k h_{[k-i]} \,g_{[i]} & \implies & 
h_{[k-s]}=\frac{1}{g_{[s]}}\Big[f_{[k]}-\sum_{i=s+1}^k h_{[k-i]} \,g_{[i]}\Big], \\
\end{eqnarray}

donde $s$ es el índice el primer término no cero del polinomio de Taylor. 

Lo que optamos por hacer primero fue calcular $(1/g)_{[k]}$ y calcular $(f/g)_{[k]}$ a partir de eso.

In [9]:
function /(b::Real, a::Taylor)
    
    n = gradomax(a);

    r = Taylor(zeros(n));
    A = prom(a,r);

    s = 1; # índice desde donde empezamos

    if A.pol[s] == 0 # checamos si el primer término no es nulo
        s += 1;
    end

    r.pol[1] = b/A.pol[s];

    for k = s:n-1
        suma = 0;
        for j = 0:k-1
            suma += r.pol[j+1]*A.pol[k-j+1]
        end

        r.pol[k+1] = (-suma)/A.pol[s];
    end
    return r
end
# Ahora definimos división entre estructuras de Taylor
/(a::Taylor, b::Taylor) = a*(1/b)

/(a::Taylor, k::Number) = Taylor(a.pol/k)
/(k::Number, a::Taylor) = Taylor(k)/a

/ (generic function with 52 methods)

In [10]:
# Muestren que su código funciona con tests adecuados; para los detalles ver 
# http://julia.readthedocs.org/en/release-0.4/stdlib/test/
using Base.Test


# Funciones de polinomios

El siguiente punto, es cómo definir funciones de polinomios. 

Como veremos aquí, esto se basará en plantear una ecuación diferencial apropiada, cuya solución es, precisamente, la expresión que estamos buscando. Este punto es *importante*, y muestra que hay una conexión importante con la solución de ecuaciones diferenciales.

Como ejemplo consideraremos la función

\begin{equation}
E(x) = \exp\big(g(x)\big),
\end{equation}

donde 

\begin{equation}
g(x) = \sum_{k=0}^\infty g_{[k]} (x-x_0)^k
\end{equation}

está escrita como una serie de Taylor (¡exacta!) alrededor de $x_0$. 


El primer punto, es que escribiremos a $E(x)$ como una serie de Taylor alrededor de $x_0$, es decir,

\begin{equation}
E(x) = \sum_{k=0}^\infty E_{[k]} (x-x_0)^k.
\end{equation}

El objetivo es determinar $E_{[k]}$ para *toda* $k$.

Dado que $E(x)$ esun polinomio en $x$, su derivada viene dada por

\begin{equation}
\frac{{\rm d} E(x)}{{\rm d}x} = \sum_{k=1}^\infty k E_{[k]}\, (x-x_0)^{k-1} .
\end{equation}

Por otra parte, la derivada de $E(x)$ en términos de $g(x)$ está dada por

\begin{equation}
\frac{{\rm d} E(x)}{{\rm d}x} = \exp\big(g(x)\big) \frac{{\rm d} g(x)}{{\rm d}x} = E(x) \frac{{\rm d} g(x)}{{\rm d}x},
\end{equation}

donde del lado derecho aparece la derivada de $g(x)$. Ya que $g(x)$ *también* está escrita en forma polinomial, su derivada es

\begin{equation}
\frac{{\rm d} g(x)}{{\rm d}x} = \sum_{k=1}^\infty k g_{[k]}\, (x-x_0)^{k-1} .
\end{equation}


Tenemos, entonces, todo lo que requerimos para escribir el lado derecho de la ecuación diferencial y explotar la aritmética de polinomios. 

\begin{eqnarray}
E(x) \frac{{\rm d} g(x)}{{\rm d}x}& = & 
\Big[ \sum_{k=0}^\infty E_{[k]} (x-x_0)^k \Big]
\Big[ \sum_{j=1}^\infty j g_{[j]} (x-x_0)^{j-1}\Big] \\
 & = & \sum_{k=1}^\infty \Big[ \sum_{j=0}^k j g_{[j]} E_{[k-j]} \; \Big] (x-x_0)^{k-1} .\\
\end{eqnarray}

La segunda línea se obtiene reordenando los términos al fijar la potencia de $(x-x_0)$, esto es, $k+j$ se toma como un nuevo índice ($k$), y el nuevo índice $j$ describe el índice del producto de los polinomios. (La potencia se deja de la forma $k-1$ ya que el lado izquierdo de la ecuación aparece así.)

Igualando con el lado izquierdo de la ecuación diferencial, que sólo involucra a la derivada de $E(x)$, tenemos que se debe cumplir

\begin{equation}
E_{[k]} = \frac{1}{k} \sum_{j=0}^k j g_{[j]} \, E_{[k-j]} = 
\frac{1}{k} \sum_{j=0}^{k} (k-j) g_{[k-j]} \, E_{[j]}, \qquad k=1,2,\dots,
\end{equation}

incluyendo *la condición inicial*

\begin{equation}
E_{[0]} = \exp\big(g(x_0)\big).
\end{equation}

Estas relaciones *de recurrencia* permiten calcular $\exp\big(g(x)\big)$, para cualquier polinomio $g(x)$.

Para el caso concreto $g(x) = x$ alrededor de $x_0=0$, donde tenemos $g_{[j]} = \delta_{j,1}$, obtenemos

\begin{eqnarray}
E_{[0]} & = & 1,\\
E_{[k]} & = & \frac{1}{k} E_{[k-1]} = \frac{1}{k(k-1)} E_{[k-2]} = \dots = \frac{1}{k!} E_{[0]} = \frac{1}{k!}\ ,
\end{eqnarray}

que es el resultado bien conocido.

### Ejercicio

Obtengan las relaciones de recurrencia para las funciones $L(x) = \log\big(g(x)\big)$, $P_\alpha(x) = \big(g(x)\big)^\alpha$, $S(x) = \sin\big(g(x)\big)$, $C(x) = \cos\big(g(x)\big)$ usando el mismo procedimiento que arriba. Implementen métodos adecuados para estas funciones en el módulo, actuando sobre estructuras `Taylor` e incluyan pruebas.

In [11]:
"""Función que evalúa un Taylor en el punto x0 (debe ser cercano al punto alrededor del que se construye el polinomio de Taylor)
"""
function evaluar(a::Taylor,x0)
    n = gradomax(a);
    ex = :(0)
    
    for k = 1:n
        ex = :($ex + $a.pol[$k]*$x0^$(k-1))
    end
    return eval(ex)
end

evaluar (generic function with 1 method)

In [24]:
import Base: exp

# Definimos el exponencial de una estructura de Taylor
function exp(a::Taylor)
    n = gradomax(a); # grado máximo
    
    exp_t = Taylor(zeros(n)); # prealocación de memoria
    exp_t.pol[1] = exp(a.pol[1]); # definimos el primer elemento de la serie
    
    for k = 2:n
        suma = 0;
        for j = 1:k
            suma += (k-j+1)*a.pol[k-j+1]*exp_t.pol[j];
        end
        exp_t.pol[k] = suma*(1/(k)); #aquí había un error pues es 1/k no 1/k-1
    end
    
    return exp_t 
end

exp (generic function with 13 methods)

Para el logaritmo hacemos un proceso similar. Tenemos a $g(x)$ dada por su expansión de Taylor alrededor de $x_0$, así:

$$L(x) = log(g(x)) = \sum_{k = 0}^{\infty} L_k(x-x_0)^k$$
$$ \Rightarrow \frac{d}{dx}L(x) = \sum_{k = 1}^{\infty} k L_k(x-x_0)^{k-1} = \frac{1}{g(x)}\frac{d}{dx}g(x) = \frac{\sum_{k = 1}^{\infty} k g_k(x-x_0)^{k-1}}{\sum_{l = 0}^{\infty}g_l(x-x_0)^l}$$ de donde se puede obtener el k-ésimo término a partir de la relación de recurrencia para la división, con la condición de que $L_0 = log(g_0)$.

In [13]:
import Base: log

function log(a::Taylor)
    n = gradomax(a); # grado máximo

    L = Taylor(zeros(n));
    L.pol[1] = log(a.pol[1]);

    for k = 1:n-1
        suma = 0;
        for j = 1: k-1
            suma += j*L.pol[j]*a.pol[k-j];
        end
        L.pol[k+1] = (1/a.pol[1])*(a.pol[k+1]-suma/k)
    end
    
    return L
end

log (generic function with 20 methods)

In [14]:
a = Taylor([2,-2]);
log(a)

Taylor{Float64}([0.6931471805599453,-1.0])

In [15]:
# Definimos la potenciación de estructuras Taylor para exponentes enteros, con metaprogramming.
#function ^(a::Taylor, n::Integer)
#    ex = :(a)
 #   k = 1;
  #  while k < n
   #     ex = :($ex * a)
    #    k += 1
   # end
    #return eval(ex)
#end
# Anoto esto porque esta función está mal definida, usaré por el momento
#que puede hacerse ^ para enteros aunque mande warning al final, por 
#ejemplo checa con [1,1,1] y debería dar [1,2,3,2,1] pero da otra cosa

In [30]:
Taylor([1,1,1])^2 #ahora este si funciona, pero necesitamos definirlo bien
#para que no haya ambigüedad porque así sólo no se puede implementar para
#por ejemplo seno y coseno

Taylor{Float64}([1.0,2.0,3.0,2.0,1.0])

In [26]:
b = Taylor([1,2,1])
exp(log(b)) #error y en general exp(b) no da lo deseado, se necesita de esto
#para poder definir ^ correctamente para no enteros

Taylor{Float64}([1.0,2.0,3.6666666666666665])

**NOTA** Los ejercicios de este notebook constituyen el contenido de la Tarea6.